In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers==2.9.0
!nvidia-smi

     |████████████████████████████████| 635 kB 5.0 MB/s 
     |████████████████████████████████| 880 kB 66.0 MB/s 
     |████████████████████████████████| 5.6 MB 29.1 MB/s 
     |████████████████████████████████| 1.2 MB 54.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=57f32e5ce1470beb0f3b01e4347efc90f2dca8c43d812f10db16cbe026362ee4
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Fri May 13 21:50:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |       

In [3]:
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(42)

In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('drive/MyDrive/outputs_imppres/model_files')

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [5]:
# optimizer
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in t5_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in t5_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-4, eps=1e-8)

In [6]:
true_false_adjective_tuples = [
                               (("Can you lend me some money?",	"I'm completely broke until payday."), "no"), 
                               (("Do you think he did the theft?",	"He is as innocent as a lamb"), "no"),
                               (("Are you sure you want the spoiler",	"haha sure. by the time, I get to it; I will forget the details, only knowing that it's touching at the end"),"yes"),
                               (("Does tomorrow afternoon work for you?",	"Yeah tomorrow afternoon works for me."),	"yes"),
                               (("oh wow, Cuban cigar? I've never smoke before",	"This one is actually from Peru, which is more my taste."),	"no"),
                               (("Are you sending them an invitation?",	"Certainly."),	"yes"),
                               (("You, uh, you don't want to go for a ride, do you?",	"Is it safe?"),	'yes'),
                               (("Have you ever made them yourself?",	"no. only eat them"),	"no"),
                               (("Have you seen my watch?",	"I will take a look for it around my house."),	"no"),
                               (("You don't look normal. Are you all right?", "I have a headache."),	"no"),
                               (("any improvements in your marathon time?",	"No, only a few minutes. I still find it difficult to run long races"),	"no"),
                               (("do you live near San Mateo?",	"no, but I'm willing to drive for good ramen"),	"no"),
                               (("Can he play the violin?",	"Can a pig fly?"),	"no"),
                               (("Aren't you scared?", "Sometimes."),	"yes")
]

In [7]:
t5_model.train()

epochs = 10

for epoch in range(epochs):
  print ("epoch ",epoch)
  for input,output in true_false_adjective_tuples:
    input_sent = "Determine the implicature: "+input[0] + input[1]+ " </s>"
    ouput_sent = output+" </s>"

    tokenized_inp = tokenizer.encode_plus(input_sent,  max_length=96, pad_to_max_length=True,return_tensors="pt")
    tokenized_output = tokenizer.encode_plus(ouput_sent, max_length=96, pad_to_max_length=True,return_tensors="pt")


    input_ids  = tokenized_inp["input_ids"]
    attention_mask = tokenized_inp["attention_mask"]

    lm_labels= tokenized_output["input_ids"]
    decoder_attention_mask=  tokenized_output["attention_mask"]


    # the forward function automatically creates the correct decoder_input_ids
    output = t5_model(input_ids=input_ids, lm_labels=lm_labels,decoder_attention_mask=decoder_attention_mask,attention_mask=attention_mask)
    loss = output[0]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

epoch  0


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9


In [8]:
import csv
file = csv.reader(open('testing.csv'))
correct = 0
wrong = 0
total = 0
tp = 0
tn = 0
fp = 0
fn = 0
for line in file:
  test_sent = "Determine the implicature: " + line[0] + "</s>"
  test_tokenized = tokenizer.encode_plus(test_sent, return_tensors="pt")

  test_input_ids  = test_tokenized["input_ids"]
  test_attention_mask = test_tokenized["attention_mask"]

  t5_model.eval()
  beam_outputs = t5_model.generate(
      input_ids=test_input_ids,attention_mask=test_attention_mask,
      max_length=64,
      early_stopping=True,
      num_beams=10,
      num_return_sequences=1,
      no_repeat_ngram_size=2
  )

  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      print (sent, line[1])
      if sent == line[1]:
        correct += 1
      else:
        wrong += 1
      if sent == 'yes' and line[1] == 'yes':
        tp += 1
      elif sent == 'yes' and line[1] == 'no':
        fp += 1
      elif sent == 'no' and line[1] == 'yes':
        fn += 1
      elif sent == 'no' and line[1] == 'no':
        tn += 1
      print(correct, wrong)
  print("")
  total += 1
  print(total)

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1432: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beam_id = beam_token_id // vocab_size


yes no
0 1

1
no no
1 1

2
no no
2 1

3
no no
3 1

4
yes no
3 2

5
yes yes
4 2

6
yes yes
5 2

7
no no
6 2

8
yes yes
7 2

9
yes yes
8 2

10
yes yes
9 2

11
yes no
9 3

12
yes no
9 4

13
no no
10 4

14
yes no
10 5

15
yes yes
11 5

16
no yes
11 6

17
yes yes
12 6

18
no yes
12 7

19
yes yes
13 7

20
yes yes
14 7

21
yes yes
15 7

22
no no
16 7

23
yes yes
17 7

24
no no
18 7

25
yes yes
19 7

26
yes no
19 8

27
no yes
19 9

28
yes yes
20 9

29
yes yes
21 9

30
no yes
21 10

31
yes yes
22 10

32
yes yes
23 10

33
yes no
23 11

34
no yes
23 12

35
no no
24 12

36
yes yse
24 13

37
yes yes
25 13

38
yes yes
26 13

39
yes yes
27 13

40
yes no
27 14

41
no no
28 14

42
no no
29 14

43
no no
30 14

44
no no
31 14

45
no yes
31 15

46
yes yes
32 15

47
yes yes
33 15

48
no no
34 15

49
yes yes
35 15

50
yes yes
36 15

51
yes yes
37 15

52
no yes
37 16

53
yes yes
38 16

54
yes yes
39 16

55
yes yes
40 16

56
yes yes
41 16

57
yes yes
42 16

58
no no
43 16

59
yes yes
44 16

60
yes yes
45 16



In [9]:
print("Correct Predictions: ",correct)
print("Incorrect Predictions:", wrong)
print("Accuracy:", correct/total)

Correct Predictions:  127
Incorrect Predictions: 55
Accuracy: 0.6978021978021978


In [10]:
recall = tp / (tp + fn)
precision = tp / (tp + fp)
f1_score = (2 * recall * precision) / (recall + precision) 
print("Recall: ", recall)
print("Precision: ", precision)
print("F1 score: ", f1_score)

Recall:  0.8055555555555556
Precision:  0.725
F1 score:  0.7631578947368423
